Original implementation of Contrastive-sc method
(https://github.com/ciortanmadalina/contrastive-sc)

In [2]:
import sys
sys.path.append("..")
import argparse
from sklearn.metrics import adjusted_rand_score, normalized_mutual_info_score
from sklearn.cluster import KMeans
from sklearn import metrics

import torch
import torch.nn as nn
import copy
from tqdm.notebook import tqdm
import models
import numpy as np
import pandas as pd
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
import st_loss

import h5py
import scipy as sp
import scanpy.api as sc
from collections import Counter
import random
import utils
import loop
import pickle

import train
import os
import glob2
plt.ion()
plt.show()
%load_ext autoreload
%autoreload 2

import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

/opt/conda/lib/python3.7/site-packages/scanpy/api/__init__.py:7: FutureWarning: 

In a future version of Scanpy, `scanpy.api` will be removed.
Simply use `import scanpy as sc` and `import scanpy.external as sce` instead.

  FutureWarning,


In [2]:
path = "../"
files = glob2.glob(f'{path}real_data/*.h5')
files = [f[len(f"'{path}real_data"):-3] for f in files]
files

['Quake_Smart-seq2_Trachea',
 'Quake_Smart-seq2_Diaphragm',
 'Quake_10x_Spleen',
 'Young',
 '10X_PBMC_select_2100',
 'mouse_ES_cell',
 'Adam',
 'Quake_10x_Bladder',
 'Quake_Smart-seq2_Lung',
 'worm_neuron_cell_select_2100',
 'Quake_10x_Limb_Muscle',
 'worm_neuron_cell',
 'mouse_bladder_cell',
 'mouse_ES_cell_select_2100',
 'Romanov',
 'Quake_Smart-seq2_Limb_Muscle',
 'Muraro',
 'mouse_bladder_cell_select_2100',
 '10X_PBMC']

In [3]:
sczi = pd.read_pickle(f"../output/pickle_results/real_data/real_data_sczi.pkl")

In [4]:
df = pd.read_pickle(f"{path}output/pickle_results/real_data_1model.pkl")

In [6]:
df.head()

,dataset,dropout,kmeans_ari_0,kmeans_nmi_0,layers,leiden_ari_0,leiden_nmi_0,lr,run,temperature
0,Quake_Smart-seq2_Trachea,0.8,0.540060,0.650050,[50],0.169197,0.517311,0.00001,0.0,0.07
1,Quake_Smart-seq2_Trachea,0.8,0.529791,0.639803,[50],0.193608,0.535134,0.00001,1.0,0.07
2,Quake_Smart-seq2_Trachea,0.8,0.488717,0.561485,"[100, 30]",0.173371,0.522036,0.00001,0.0,0.07
3,Quake_Smart-seq2_Trachea,0.8,0.466333,0.543368,"[100, 30]",0.190239,0.530519,0.00001,1.0,0.07
4,Quake_Smart-seq2_Trachea,0.8,0.547574,0.672945,"[200, 100, 30]",0.189931,0.536097,0.00001,0.0,0.07


In [ ]:
# df = pd.DataFrame()

for dataset in files[2:]:

    print(f">>>>> Data {dataset}")
    print("SCZI ", sczi[sczi["dataset"] == dataset]["ARI"].mean())
    data_mat = h5py.File(f"{path}real_data/{dataset}.h5", "r")
    X = np.array(data_mat['X'])
    Y = np.array(data_mat['Y'])

    cluster_number = np.unique(Y).shape[0]
    nb_genes = 1500
    data_mat = h5py.File(f"{path}real_data/{dataset}.h5", "r")
    X = np.array(data_mat['X'])
    Y = np.array(data_mat['Y'])

    X = train.preprocess(X, nb_genes=nb_genes)
    for temperature in tqdm([0.07, 0.5, 0.01]):
        for dropout in [ 0.8, 0.2, 0.5, 0.9]:
            for lr in [1e-5, 1e-4, 1e-6]:
                for layers in [[50],[100, 30],  [200, 100, 30],  [200, 100, 30, 30]]:
                    for run in range(2):
                        torch.manual_seed(run)
                        torch.cuda.manual_seed_all(run)
                        np.random.seed(run)
                        random.seed(run)
                        torch.backends.cudnn.deterministic = True
                        torch.backends.cudnn.benchmark = False
                        dresults = train.run(X,
                                             cluster_number,
                                             dataset,
                                             Y=Y,
                                             nb_epochs=20,
                                             lr=lr,
                                             temperature=temperature,
                                             dropout=dropout,
                                             evaluate=False,
                                             n_ensemble=1,
                                             layers=layers,
                                             save_to=f"{path}output/real_data/{run}/",
                                             save_pred = False)
                        dresults["temperature"] = temperature
                        dresults["dropout"] = dropout
                        dresults["lr"] = lr
                        dresults["layers"] = str(layers)
                        dresults["run"] = run
                        print(f".", end = "")
#                         print(f"# {temperature}, {dropout}, {lr}, {layers}", 
#                               dresults.get('COMBINED_kmeans_ari', ""),
#                               dresults.get('COMBINED_leiden_ari', ""), dresults.get('kmeans_ari_0',""),
#                               dresults.get('leiden_ari_0', ""))
                        df = df.append(dresults, ignore_index=True)

                        df.to_pickle(f"{path}output/pickle_results/real_data_1model.pkl")

>>>>> Data Quake_10x_Spleen
SCZI  0.909534824400645
(9552, 23341) (9552, 23341) keeping 1500 genes


................................................................................................................................................................................................................................................................................................
>>>>> Data Young
SCZI  0.6629811886776039
(5685, 33658) (5685, 33658) keeping 1500 genes


................................................................................................................................................................................................................................................................................................
>>>>> Data 10X_PBMC_select_2100
SCZI  0.597949963742046
(2100, 16653) (2100, 16653) keeping 1500 genes


................................................................................................................................................................................................................................................................................................
>>>>> Data mouse_ES_cell
SCZI  0.800376096099337
(2717, 24175) (2717, 24175) keeping 1500 genes


................................................................................................................................................................................................................................................................................................
>>>>> Data Adam
SCZI  0.8634561030635544
(3660, 23797) (3660, 23797) keeping 1500 genes


..................

In [ ]:
str(layers)

In [9]:
df = pd.read_pickle(f"../output/pickle_results/real_data_1model.pkl")
df.groupby(["temperature", "layers", "dropout", "lr"])["kmeans_ari_0"].mean().unstack(["layers", "lr"])

layers              [100, 30]                     [200, 100, 30, 30]  \
lr                   0.000001  0.000010  0.000100           0.000001   
temperature dropout                                                    
0.01        0.2      0.516109  0.516109  0.516109           0.433314   
            0.5      0.577762  0.577762  0.577762           0.556851   
            0.8      0.657434  0.657434  0.657434           0.641832   
            0.9      0.694508  0.694508  0.694508           0.614636   
0.07        0.2      0.278396  0.278396  0.278396           0.239343   
            0.5      0.496646  0.496646  0.496646           0.433621   
            0.8      0.641900  0.641900  0.641900           0.674056   
            0.9      0.680961  0.680961  0.680961           0.693146   
0.50        0.2      0.405372  0.423459  0.422841           0.388501   
            0.5      0.467338  0.467338  0.467338           0.438229   
            0.8      0.608725  0.608725  0.608725           0.585777   
            0.9      0.638594  0.638594  0.638594           0.622103   

layers                                  [200, 100, 30]                      \
lr                   0.000010  0.000100       0.000001  0.000010  0.000100   
temperature dropout                                                          
0.01        0.2      0.433314  0.433314       0.477152  0.477152  0.477152   
            0.5      0.556851  0.556851       0.512568  0.512568  0.512568   
            0.8      0.641832  0.641832       0.653073  0.653073  0.653073   
            0.9      0.614636  0.614636       0.668815  0.668815  0.668815   
0.07        0.2      0.239343  0.239343       0.235293  0.235293  0.235293   
            0.5      0.433621  0.433621       0.471576  0.471576  0.471576   
            0.8      0.674056  0.674056       0.672430  0.672430  0.672430   
            0.9      0.693146  0.693146       0.657798  0.657798  0.657798   
0.50        0.2      0.399817  0.388501       0.416959  0.441847  0.416959   
            0.5      0.438229  0.438229       0.493725  0.493725  0.493725   
            0.8      0.585777  0.585777       0.601673  0.601673  0.601673   
            0.9      0.622103  0.622103       0.621916  0.621916  0.621916   

layers                   [50]                      
lr                   0.000001  0.000010  0.000100  
temperature dropout                                
0.01        0.2      0.570401  0.570401  0.570401  
            0.5      0.597860  0.597860  0.597860  
            0.8      0.704679  0.704679  0.704679  
            0.9      0.778514  0.778514  0.778514  
0.07        0.2      0.336452  0.336452  0.336452  
            0.5      0.503766  0.503766  0.503766  
            0.8      0.611546  0.611546  0.611546  
            0.9      0.686385  0.686385  0.686385  
0.50        0.2      0.546973  0.549111  0.549111  
            0.5      0.555599  0.555599  0.555599  
            0.8      0.671923  0.671923  0.671923  
            0.9      0.666699  0.666699  0.666699

In [10]:
df = pd.read_pickle(f"../output/pickle_results/real_data_1model.pkl")
df.groupby(["temperature", "layers", "dropout", "lr"])["kmeans_ari_0"].mean().unstack(["layers", "lr"]).max()

layers              lr      
[100, 30]           0.000001    0.694508
                    0.000010    0.694508
                    0.000100    0.694508
[200, 100, 30, 30]  0.000001    0.693146
                    0.000010    0.693146
                    0.000100    0.693146
[200, 100, 30]      0.000001    0.672430
                    0.000010    0.672430
                    0.000100    0.672430
[50]                0.000001    0.778514
                    0.000010    0.778514
                    0.000100    0.778514
dtype: float64

In [11]:
sczi = pd.read_pickle(f"../output/pickle_results/real_data/real_data_sczi.pkl")

In [13]:
sczi[sczi["dataset"].isin(df["dataset"].values)].mean()

ARI       0.803297
NMI       0.802730
sil       0.071988
run       1.000000
time    185.308213
dtype: float64

In [ ]:
run = 0 
dataset = 'Quake_Smart-seq2_Diaphragm'
dataset = 'Quake_Smart-seq2_Trachea'

In [ ]:
torch.manual_seed(run)
torch.cuda.manual_seed_all(run)
np.random.seed(run)
random.seed(run)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
print(f">>>>> Data {dataset}")

data_mat = h5py.File(f"{path}real_data/{dataset}.h5", "r")
X = np.array(data_mat['X'])
Y = np.array(data_mat['Y'])
print(np.where(X == 0)[0].shape[0] / (X.shape[0] * X.shape[1]))
cluster_number = np.unique(Y).shape[0]
nb_genes = 1500
data_mat = h5py.File(f"{path}real_data/{dataset}.h5", "r")
X = np.array(data_mat['X'])
Y = np.array(data_mat['Y'])

X = train.preprocess(X, nb_genes=nb_genes)

In [ ]:
run = 0
torch.manual_seed(run)
torch.cuda.manual_seed_all(run)
np.random.seed(run)

torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False


In [ ]:
res = train.run(X,
        cluster_number,
        dataset,
        Y=Y,
        nb_epochs=20,
        lr=1e-5,
        temperature=0.07,
        dropout=0.8,
        evaluate=True,
        n_ensemble=3,
        layers = [256, 64, 32],
        save_to=f"{path}output/real_data/{run}/")

In [ ]:
res.keys()

In [ ]:
dresults['COMBINED_kmeans_ari'], dresults['COMBINED_leiden_ari'], dresults['kmeans_ari_0'], dresults['leiden_ari_0']

In [ ]:
df = df.append(res, ignore_index=True)

In [ ]:
df

In [ ]:
df.groupby("dataset").mean()

# Importance of input size

In [ ]:
df = pd.DataFrame(
    columns=["dataset", "perc0", "nb_genes", "exp", "ari", "run"])
print(df.shape)
for dataset in files:

    print(f">>>>> Data {dataset}")

    data_mat = h5py.File(f"{path}real_data/{dataset}.h5", "r")
    for run in range(2):
        torch.manual_seed(run)
        torch.cuda.manual_seed_all(run)
        np.random.seed(run)
        random.seed(run)
        torch.backends.cudnn.deterministic = True
        torch.backends.cudnn.benchmark = False
        for nb_genes in [100, 200, 500, 1000, 1500, 2000, 5000, 8000]:

            X = np.array(data_mat['X'])
            Y = np.array(data_mat['Y'])
            perc_0 = np.where(X == 0)[0].shape[0] / (X.shape[0] * X.shape[1])
            print(f"Perc 0 {perc_0}")
            cluster_number = np.unique(Y).shape[0]

            X = train.preprocess(X, nb_genes=nb_genes)
            nb_zeros = int(0.8 * nb_genes)
            dresults = train.train(
                X,
                cluster_number,
                dataset,
                Y,
                n_ensemble=1,
                epochs=100,
                nb_zeros=nb_zeros,
                save_to=f"{path}output/real_data/inputs/{dataset}_{nb_genes}/")

            #         df.loc[df.shape[0]] = [
            #                 dataset, perc_0, nb_genes, 'kmeans_representation_0',dresults['kmeans_representation_0']
            #             ]
            df.loc[df.shape[0]] = [
                dataset, perc_0, nb_genes, 'leiden_representation_0',
                dresults['leiden_representation_0'], run]

#             pxt = PCA(2).fit_transform(X)
#             dresults["original"] = utils.evaluate(X, Y, cluster_number)[1]
#             dresults["pca"] = utils.evaluate(pxt, Y, cluster_number)[1]
            print(dresults)
    df.to_pickle(f"{path}output/pickle_results/real_data_input_size.pkl")

In [ ]:
df = pd.read_pickle(f"{path}output/pickle_results/real_data_input_size.pkl")

In [ ]:
df.groupby(["nb_genes"]).mean()

In [ ]:
dataset_names = {
    '10X_PBMC': '10X PBMC',
    '10X_PBMC_select_2100': '10X PBMC (2100)',
    'mouse_ES_cell': 'Mouse ES\nCell',
    'mouse_ES_cell_select_2100': 'Mouse ES\nCell (2100)',
    'worm_neuron_cell_select_2100': 'Worm Neuron\nCell (2100)',
    'worm_neuron_cell': 'Worm Neuron\nCell',
    'mouse_bladder_cell': 'Mouse Bladder\nCell',
    'mouse_bladder_cell_select_2100': 'Mouse Bladder\n Cell (2100)'
}

df["dataset"] = df["dataset"].apply(lambda x: dataset_names[x])

df = df.rename(columns = {"nb_genes": "Nb input genes"})

In [ ]:
df["dataset"].unique()


In [ ]:
import seaborn as sns
datasets = ['10X PBMC',  'Mouse ES\nCell','Worm Neuron\nCell', 'Mouse Bladder\nCell']
plt.figure(figsize=(10, 3.3))
ax = plt.subplot(111)
sns.barplot(
    hue="Nb input genes",
    y="ari",
    x="dataset",
    data=df[df["dataset"].isin(datasets)],
    ax=ax,
    edgecolor='black',
    linewidth=1.5,
)
plt.ylabel("ARI")
plt.xlabel("")
plt.legend(title= "Nb input genes",bbox_to_anchor=(1, 1))
sns.despine()
plt.tight_layout()

plt.savefig(f"{path}diagrams/real_input_size.pdf", bbox_inches='tight')

In [ ]:
datasets = ['10X PBMC (2100)',
       'Mouse ES\nCell (2100)', 'Worm Neuron\nCell (2100)',
       'Mouse Bladder\n Cell (2100)']
plt.figure(figsize=(10, 3.3))
ax = plt.subplot(111)
sns.barplot(
    hue="Nb input genes",
    y="ari",
    x="dataset",
    data=df[df["dataset"].isin(datasets)],
    ax=ax,
    edgecolor='black',
    linewidth=1.5,
)
plt.ylabel("ARI")
plt.xlabel("")
plt.legend(title= "Nb input genes",bbox_to_anchor=(1, 1))
sns.despine()
plt.tight_layout()

plt.savefig(f"{path}diagrams/real_input_size_2100.pdf", bbox_inches='tight')

# Importance of the number of epochs

In [ ]:
df = pd.DataFrame(
    columns=["dataset", "perc0", "nb_epochs", "exp", "ari", "run"])
print(df.shape)
for dataset in files:

    print(f">>>>> Data {dataset}")

    data_mat = h5py.File(f"{path}real_data/{dataset}.h5", "r")
    nb_genes = 1500
    for epochs in [5, 50, 100, 300]:

        X = np.array(data_mat['X'])
        Y = np.array(data_mat['Y'])
        perc_0 = np.where(X == 0)[0].shape[0] / (X.shape[0] * X.shape[1])
        print(f"Perc 0 {perc_0}")
        cluster_number = np.unique(Y).shape[0]

        X = train.preprocess(X, nb_genes=nb_genes)
        nb_zeros = int(0.8 * nb_genes)
        for run in range(2):
            torch.manual_seed(run)
            torch.cuda.manual_seed_all(run)
            np.random.seed(run)
            random.seed(run)
            torch.backends.cudnn.deterministic = True
            torch.backends.cudnn.benchmark = False

            dresults = train.train(
                X,
                cluster_number,
                dataset,
                Y,
                n_ensemble=1,
                epochs=epochs,
                nb_zeros=nb_zeros,
                save_to=f"{path}output/real_data/epochs/{dataset}_{epochs}/")

            df.loc[df.shape[0]] = [
                dataset, perc_0, epochs, 'kmeans_representation_0',
                dresults['kmeans_representation_0'], run
            ]
            df.loc[df.shape[0]] = [
                dataset, perc_0, epochs, 'leiden_representation_0',
                dresults['leiden_representation_0'], run
            ]

            print(dresults)
            df.to_pickle(f"{path}output/pickle_results/real_data_epochs.pkl")

In [ ]:
df = pd.read_pickle(f"{path}output/pickle_results/real_data_epochs.pkl")

In [ ]:
dataset_names = {
    '10X_PBMC': '10X PBMC',
    '10X_PBMC_select_2100': '10X PBMC (2100)',
    'mouse_ES_cell': 'Mouse ES\nCell',
    'mouse_ES_cell_select_2100': 'Mouse ES\nCell (2100)',
    'worm_neuron_cell_select_2100': 'Worm Neuron\nCell (2100)',
    'worm_neuron_cell': 'Worm Neuron\nCell',
    'mouse_bladder_cell': 'Mouse Bladder\nCell',
    'mouse_bladder_cell_select_2100': 'Mouse Bladder\n Cell (2100)'
}

df["dataset"] = df["dataset"].apply(lambda x: dataset_names[x])

df = df.rename(columns = {"nb_epochs": "Nb epochs"})

In [ ]:
import seaborn as sns
datasets = ['10X PBMC',  'Mouse ES\nCell','Worm Neuron\nCell', 'Mouse Bladder\nCell']
plt.figure(figsize=(7, 3))
ax = plt.subplot(111)
sns.barplot(
    hue="Nb epochs",
    y="ari",
    x="dataset",
    data=df[df["dataset"].isin(datasets)],
    ax=ax,
    edgecolor='black',
    linewidth=1.5,
)
plt.ylabel("ARI")
plt.xlabel("")
plt.legend(title ="Number of epochs",bbox_to_anchor=(1, 1))
sns.despine()
plt.tight_layout()

plt.savefig(f"{path}diagrams/real_nb_epochs.pdf", bbox_inches='tight')